In [ ]:
import pickle
from collections import namedtuple
Peiling = namedtuple('Peiling', ['verwacht', 'laag', 'hoog'])

In [ ]:
with open('peilingen.pkl', 'rb') as fh:
    numbers = pickle.load(fh)

# Coalitiewijzer
Druk op de knoppen om partijen toe te voegen aan je coalitie. Een meerderheid heeft 76 zetels nodig. De helder rode zetel is het verwachte aantal zetels. In licht rood zie je onder- en bovengrenzen van het verwachte aantal zetels op basis van verschillende peilingen.

Press the buttons to add parties to your coalition. A majority coalition needs 76 seats. The bright red seat is the expected number of seats. In light red we show the lower and upper limits of the expected amount of seats based on different polls.

Data: https://peilingwijzer.tomlouwerse.nl/p/laatste-cijfers.html

In [ ]:
import ipywidgets as widgets
from ipyvuetify import VuetifyTemplate, Btn
import traitlets

In [ ]:
class SeatsWidget(VuetifyTemplate):
    rows = traitlets.Int(5).tag(sync=True)
    cols = traitlets.Int(30).tag(sync=True)
    low = traitlets.Int(0).tag(sync=True)
    exp = traitlets.Int(0).tag(sync=True)
    high = traitlets.Int(0).tag(sync=True)
    template = traitlets.Unicode('''
<template>
    <div>
        <table>
        <tr v-for="i in rows">
            <td v-for="j in cols/2">
                <div>
                    <v-icon v-if="((j-1) * rows + i) < low"        style="color: #666666">mdi-seat</v-icon>
                    <v-icon v-else-if="((j-1) * rows + i) < exp"   style="color: red; opacity: 30%; transform: rotate(90deg)">mdi-seat</v-icon>
                    <v-icon v-else-if="((j-1) * rows + i) == exp"  style="color: red">mdi-seat</v-icon>
                    <v-icon v-else-if="((j-1) * rows + i) <= high" style="color: red; opacity: 30%; transform: rotate(-90deg)">mdi-seat</v-icon>
                    <v-icon v-else style="color: #bbbbbb; transform: rotate(180deg)">mdi-seat</v-icon>
                </div>
            </td>
            <td>
                <div>
                    <v-icon style="color: blue">mdi-dots-vertical</v-icon>
                </div>
            </td>
            <td v-for="j in cols/2">
                <div>
                    <v-icon v-if="((j + cols/2 - 1) * rows + i) < low"        style="color: #666666">mdi-seat</v-icon>
                    <v-icon v-else-if="((j + cols/2 - 1) * rows + i) < exp"   style="color: red; opacity: 30%; transform: rotate(90deg)">mdi-seat</v-icon>
                    <v-icon v-else-if="((j + cols/2 - 1) * rows + i) == exp"  style="color: red">mdi-seat</v-icon>
                    <v-icon v-else-if="((j + cols/2 - 1) * rows + i) <= high" style="color: red; opacity: 30%; transform: rotate(-90deg)">mdi-seat</v-icon>
                    <v-icon v-else style="color: #bbbbbb; transform: rotate(180deg)">mdi-seat</v-icon>
                </div>
            </td>
        </tr>
    </div>
</template>
''').tag(sync=True)
    
seats = SeatsWidget()

In [ ]:
toggle_buttons = {key: widgets.ToggleButton(description=key, value=False,
                                            layout=widgets.Layout(height='auto', width='auto'))
                  for key in numbers.keys()}

grid_width = 5

grid = widgets.GridspecLayout((len(numbers) - 1) // grid_width + 1, grid_width)
for ix, (key, button) in enumerate(toggle_buttons.items()):
    ix1, ix2 = ix // grid_width, ix % grid_width
    grid[ix1, ix2] = button

In [ ]:
def on_toggle(**toggles):
    selected = [partij for partij, partij_selected in toggles.items() if partij_selected]

    seats.low = int(sum(numbers[partij].laag for partij in selected))
    seats.exp = int(sum(numbers[partij].verwacht for partij in selected))
    seats.high = int(sum(numbers[partij].hoog for partij in selected))

    #     print(f"verwacht aantal zetels: {expectation_value} (laag: {low}, hoog: {high})")

interact_out = widgets.interactive_output(on_toggle, toggle_buttons)
display(seats)
display(grid, interact_out)